# Pandas EDA Cheat Sheet — Live Examples

A hands‑on notebook to learn/recall the most useful **pandas** patterns for Exploratory Data Analysis (EDA).
Run cells top‑to‑bottom; each section is self‑contained with small examples.

**Contents**
1. Setup
2. Core Objects & Creation
3. IO (Read/Write)
4. Quick EDA Starters
5. Selecting & Filtering
6. Columns (Create/Rename/Drop/Reorder)
7. Missing Values
8. Types & Conversions
9. Sorting, Dedup, Sampling
10. Aggregation & GroupBy
11. Pivoting/Crosstab/Reshape
12. Joins & Concatenation
13. Dates & Times
14. String Operations
15. Window Functions
16. Binning & Ranking
17. MultiIndex Essentials
18. Outliers & Clipping
19. Performance & Memory Tips
20. EDA Starter Function
21. Quick Plots
22. Common Gotchas & Patterns

> Tip: Use `Shift+Enter` to execute a cell.


## 1) Setup

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display tweaks
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.float_format", lambda v: f"{v:,.3f}")

print(pd.__version__)


### Create some reusable sample data

In [ ]:

# Deterministic random
rng = np.random.default_rng(42)

# A small 'facts' table with numeric + categorical + dates
n = 200
dates = pd.date_range("2024-01-01", periods=n, freq="D")
cities = np.array(["Mumbai", "Delhi", "Bengaluru", "Chennai", "Pune"], dtype=object)
labels = np.array(list("ABC"), dtype=object)

df = pd.DataFrame({
    "id": np.arange(1, n+1),
    "date": dates,
    "city": rng.choice(cities, size=n),
    "label": rng.choice(labels, size=n, p=[0.5, 0.3, 0.2]),
    "units": rng.integers(1, 50, size=n),
    "price": rng.normal(loc=100.0, scale=15.0, size=n).round(2)
})
df["revenue"] = (df["units"] * df["price"]).round(2)

# Create a small dimension table for joins
dim_city = pd.DataFrame({
    "city": cities,
    "region": ["West", "North", "South", "South", "West"],
    "population_millions": [12.5, 19.0, 13.1, 11.2, 7.1]
})

# Introduce some missing values and text noise
df.loc[rng.choice(df.index, size=10, replace=False), "price"] = np.nan
df.loc[rng.choice(df.index, size=5, replace=False), "city"] = None
df.loc[rng.choice(df.index, size=5, replace=False), "label"] = None
df.loc[rng.choice(df.index, size=5, replace=False), "units"] = np.nan

df.head()


## 2) Core Objects & Creation

In [ ]:

# Series
s = pd.Series([1, 2, 3], name="s")
display(s)

# DataFrame from dict of lists
df_basic = pd.DataFrame({"A": [1, 2, 3], "B": ["x", "y", "z"]})
display(df_basic)

# From list of dicts
rows = [{"id": 1, "val": 10}, {"id": 2, "val": 20}]
df_records = pd.DataFrame(rows)
display(df_records)

# From NumPy array
arr = np.arange(12).reshape(3, 4)
df_from_arr = pd.DataFrame(arr, columns=list("ABCD"))
display(df_from_arr)

# Empty frame
df_empty = pd.DataFrame(columns=["id", "name", "score"])
display(df_empty)


## 3) IO: Read & Write

In [ ]:

# Write our sample df to CSV and read it back
csv_path = "sample_sales.csv"
df.to_csv(csv_path, index=False)
df_csv = pd.read_csv(csv_path, parse_dates=["date"], dtype={"id": "Int64"})
display(df_csv.head())

# JSON records (lines)
json_path = "sample_sales.jsonl"
df.to_json(json_path, orient="records", lines=True, date_format="iso")
df_json = pd.read_json(json_path, orient="records", lines=True)
display(df_json.head())

print(f"Saved example files to: {csv_path} and {json_path}")


## 4) Quick EDA Starters

In [ ]:

display(df.shape)
df.info()
display(df.dtypes)
display(df.head(5))
display(df.tail(3))
display(df.sample(5, random_state=0))

display(df.describe(datetime_is_numeric=True))
display(df.nunique())
display(df.isna().sum())
display(df.memory_usage(deep=True))

# Correlation on numeric cols
display(df.select_dtypes("number").corr(numeric_only=True, method="pearson"))


In [ ]:

# Column overview helper
overview = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_unique": df.nunique(),
    "%missing": (df.isna().mean() * 100).round(2),
})
display(overview)


## 5) Selecting & Filtering

In [ ]:

# Columns
display(df["city"].head())
display(df[["date", "city", "revenue"]].head())

# Row/Column by label/position
print(df.loc[5, "city"], df.iloc[5, 2])

# Boolean filters
mask = (df["units"] > 25) & (df["city"].isin(["Mumbai", "Delhi"]))
display(df[mask].head())

# query/eval
display(df.query("units > 25 and city in ['Mumbai', 'Delhi']").head())
tmp = df.copy()
tmp.eval("rev_per_unit = revenue / units", inplace=True)
display(tmp.head())


## 6) Columns — Create, Rename, Drop, Reorder

In [ ]:

tmp = df.copy()
tmp["unit_bucket"] = pd.cut(tmp["units"], bins=[0,10,20,30,50], include_lowest=True)
tmp = tmp.assign(gm=lambda d: (d["revenue"] / d["units"]).round(2))

tmp = tmp.rename(columns={"gm": "gross_margin"})
tmp = tmp.drop(columns=["unit_bucket"])

# Reorder with id/date first
cols = ["id", "date", *[c for c in tmp.columns if c not in ["id","date"]]]
tmp = tmp[cols]
display(tmp.head())


## 7) Missing Values

In [ ]:

display(df.isna().sum())

mfix = df.copy()
mfix["units"] = mfix["units"].fillna(0)
mfix["price"] = mfix["price"].ffill().bfill()
mfix = mfix.dropna(subset=["city"])  # drop rows where city is NA
display(mfix.isna().sum())


## 8) Types & Conversions

In [ ]:

tmp = df.copy()
tmp = tmp.astype({"id": "Int64"})
tmp["date"] = pd.to_datetime(tmp["date"], errors="coerce")
tmp["city"] = tmp["city"].astype("category")
display(tmp.dtypes)

num = tmp.select_dtypes(include="number")
obj = tmp.select_dtypes(include="object")
display(num.head(2))
display(obj.head(2))

# Categorical with explicit order
tmp["label"] = tmp["label"].astype("category")
tmp["label"] = tmp["label"].cat.set_categories(list("ABC"), ordered=True)
display(tmp[["label"]].head(5))


## 9) Sorting, Dedup, Sampling

In [ ]:

tmp = df.sort_values(["city","revenue"], ascending=[True, False])
display(tmp.head())

dupes = df.duplicated(subset=["date", "city"], keep="first")
display(dupes.value_counts())

deduped = df.drop_duplicates(subset=["date", "city"], keep="last")
display(deduped.head())

# Stratified sample: 10% per label (dropping NA labels)
strat = (df.dropna(subset=["label"])
           .groupby("label", group_keys=False)
           .apply(lambda g: g.sample(frac=0.1, random_state=42)))
display(strat.head())


## 10) Aggregation & GroupBy

In [ ]:

display(df["units"].sum())
display(df["price"].mean())
display(df["label"].value_counts(dropna=False))

display(df.agg({"units": ["min","max","mean","median"],
                "price": ["count","nunique"]}))

g = df.groupby(["city", "label"], dropna=False, as_index=False)
display(g["revenue"].agg(["count","mean","sum","min","max"]).head())

tmp = df.copy()
tmp["rev_z"] = tmp.groupby("city")["revenue"].transform(lambda x: (x - x.mean()) / x.std(ddof=0))
tmp["city_avg_rev"] = tmp.groupby("city")["revenue"].transform("mean")
display(tmp.head())


## 11) Pivoting, Crosstab, Reshape

In [ ]:

pt = pd.pivot_table(df, index="city", columns="label",
                    values="revenue", aggfunc="mean", fill_value=0, margins=True)
display(pt)

ct = pd.crosstab(df["city"], df["label"], dropna=False)
display(ct)

long = df.melt(id_vars=["id","city"], value_vars=["units","price","revenue"],
               var_name="metric", value_name="value")
display(long.head())

wide = long.set_index(["id","metric"]).unstack("metric")
display(wide.head())

long2 = wide.stack("metric").reset_index()
display(long2.head())


## 12) Joins & Concatenation

In [ ]:

m = pd.merge(df, dim_city, on="city", how="left", indicator=True, validate="m:1")
display(m.head())
display(m["_merge"].value_counts())

# Concatenate rows
rows = pd.concat([df.head(3), df.tail(3)], axis=0, ignore_index=True)
display(rows)

# Concatenate columns (demo with a small selection)
cols = pd.concat([df[["id","city"]].head(5).reset_index(drop=True),
                  df[["units","price"]].head(5).reset_index(drop=True)], axis=1)
display(cols)


## 13) Dates & Times

In [ ]:

tmp = df.copy()
tmp["ts"] = pd.to_datetime(tmp["date"], utc=True)
tmp["year"] = tmp["ts"].dt.year
tmp["dow"] = tmp["ts"].dt.day_name()
tmp["hour"] = 9  # pretend business hour
display(tmp[["ts","year","dow","hour"]].head())

daily_rev = (tmp.set_index("ts")
               .resample("D")["revenue"]
               .sum())
display(daily_rev.head())


## 14) String Operations

In [ ]:

names = pd.Series(["  Alice  ", "Bob", "carol  ", None, "AL-1234"])
display(names.str.len())
display(names.str.lower().str.strip())
display(names.str.contains(r"^a", case=False, na=False))
display(names.str.replace(r"\s+", " ", regex=True))
display(names.str.extract(r"(\d{4})", expand=False))
display(names.str.split("-", n=1, expand=True))


## 15) Window Functions (Rolling/Expanding/EWM)

In [ ]:

tmp = df.sort_values("date").copy()
tmp["roll_mean_7"] = tmp["revenue"].rolling(window=7, min_periods=1).mean()
tmp["roll_sum_30"] = tmp["revenue"].rolling(30, min_periods=5).sum()
tmp["cum_mean"] = tmp["revenue"].expanding().mean()
tmp["ema_0_2"] = tmp["revenue"].ewm(alpha=0.2, adjust=False).mean()
display(tmp[["date","revenue","roll_mean_7","roll_sum_30","cum_mean","ema_0_2"]].head(12))


## 16) Binning & Ranking

In [ ]:

tmp = df.copy()
tmp["units_bin"] = pd.cut(tmp["units"], bins=[0,10,20,30,50], labels=["0-10","11-20","21-30","31-50"], include_lowest=True)
tmp["rev_q"] = pd.qcut(tmp["revenue"], q=4, labels=["Q1","Q2","Q3","Q4"])
tmp["rank_desc"] = tmp["revenue"].rank(method="dense", ascending=False)
display(tmp[["units","units_bin","revenue","rev_q","rank_desc"]].head())


## 17) MultiIndex Essentials

In [ ]:

mi = df.set_index(["city","date"]).sort_index()
# Select by tuple
sample = mi.loc[mi.index[0]]
display(sample.head(3))

# Cross-section
if "Mumbai" in mi.index.get_level_values("city"):
    display(mi.xs("Mumbai", level="city").head(3))

# Swap, then reset
mi2 = mi.swaplevel("city","date").sort_index()
display(mi2.head(3))
display(mi2.reset_index().head(3))


## 18) Outliers & Clipping

In [ ]:

lo, hi = df["revenue"].quantile([0.01, 0.99])
tmp = df.copy()
tmp["rev_clipped"] = tmp["revenue"].clip(lower=lo, upper=hi)

z = (tmp["revenue"] - tmp["revenue"].mean()) / tmp["revenue"].std(ddof=0)
tmp["rev_outlier"] = (z.abs() > 3)
display(tmp[["revenue","rev_clipped","rev_outlier"]].head(10))
tmp["rev_outlier"].value_counts()


## 19) Performance & Memory Tips

In [ ]:

tmp = df.copy()
num_cols = tmp.select_dtypes(include="number").columns
tmp[num_cols] = tmp[num_cols].apply(pd.to_numeric, downcast="integer")
tmp[num_cols] = tmp[num_cols].apply(pd.to_numeric, downcast="float")
before_mb = df.memory_usage(deep=True).sum() / (1024**2)
after_mb  = tmp.memory_usage(deep=True).sum() / (1024**2)
print(f"Before: {before_mb:.3f} MB, After: {after_mb:.3f} MB")

# Make low-cardinality strings categorical
tmp2 = df.copy()
for c in tmp2.select_dtypes(include="object").columns:
    if tmp2[c].nunique(dropna=False) / len(tmp2) < 0.5:
        tmp2[c] = tmp2[c].astype("category")

print(tmp2.dtypes)


## 20) EDA Starter Function

In [ ]:

def eda_overview(d):
    out = {}
    out["shape"] = d.shape
    out["dtypes"] = d.dtypes.astype(str).to_dict()
    out["missing_pct"] = (d.isna().mean().round(3) * 100).to_dict()
    out["nunique"] = d.nunique().to_dict()
    out["numeric_describe"] = d.select_dtypes("number").describe().round(3)
    out["object_top_values"] = {c: d[c].value_counts().head(5).to_dict()
                                for c in d.select_dtypes(include=["object","category"]).columns}
    return out

summary = eda_overview(df)
summary["shape"], list(summary["dtypes"].items())[:5]


## 21) Quick Plots (matplotlib via pandas)

In [ ]:

# Histogram
df["revenue"].hist(bins=30); plt.title("Revenue"); plt.xlabel("Revenue"); plt.ylabel("Frequency"); plt.show()


In [ ]:

# Boxplot by group
df.boxplot(column="revenue", by="city", rot=45); plt.tight_layout(); plt.suptitle(""); plt.title("Revenue by City"); plt.show()


In [ ]:

# Scatter
df.plot(kind="scatter", x="units", y="price"); plt.title("Units vs Price"); plt.show()


In [ ]:

# KDE
df["price"].plot(kind="kde"); plt.title("Price Density"); plt.xlabel("Price"); plt.show()


## 22) Common Gotchas & Patterns

- **SettingWithCopy**: prefer `df.loc[mask, "col"] = value` (avoid chained indexing).
- **Mixed dtypes**: clean strings like `"1,234"` via `str.replace(",", "")` then `pd.to_numeric(..., errors="coerce")`.
- **Dates**: always `pd.to_datetime(..., errors="coerce")` and check `isna()` after.
- **Duplicate keys before join**: use `validate="1:1"` or `validate="m:1"` in `merge` to catch surprises.
- **Large CSVs**: use `usecols=`, `dtype=`, `parse_dates=`, `chunksize=`, or prefer **Parquet** for speed/size.
- **Categoricals**: for low‑cardinality strings to save memory & speed up `groupby`/`merge`.
- **Vectorize**: prefer vectorized ops (`.map`, `.merge`, `.assign`, `.eval`) instead of Python loops.


_Generated on 2025-09-13 15:16:47_